This part of code defines 3 functions:
* handle_query_condition:
      In the corresponding notebook single-db-resource-consumption, we pass the values of DB_CONN_ID, END_TIME, INTERVAL, REPORT_TYPE variables to "singleDBCondition" as a string. This function is to split "singleDBCondition" and assign the value to each variable. 
* get_original_datatime_str:
      In the query result, we retrieve the date and hour information from Timestamp data type by using the Db2 functions Date() and Hour(). The format of date and hour is converted to like "2018-01-01 01:00:00". This information is used to fill the missing datetime in the graph drawing.
* format_x_axis:
      To match datetime as the label to ticks of x-Axis.

In [ ]:
# import required lib
import datetime
import re
import time

'''
In corresponding notebook single-db-resource-consumption, 
we pass value of DB_CONN_ID, END_TIME, INTERVAL, REPORT_TYPE to "singleDBCondition" as a String
This function is to split "singleDBCondition" and assign the values to variables 
'''
DB_CONN_ID = ''
END_TIME = ''
INTERVAL = 0
START_TIME = ''
REPORT_TYPE = ''
ISOK_ALL_PARA = 1
GENERAL_ERROR = None

def handle_query_condition():
    # declare global variables
    global DB_CONN_ID
    global END_TIME
    global INTERVAL
    global START_TIME
    global REPORT_TYPE
    global ISOK_ALL_PARA
    global GENERAL_ERROR
    
    # read parameters from singleDBCondition
    where_cause = !cat singleDBCondition
    where_cause = re.search(r'DB_CONN_ID\s?=\s?.+END_TIME\s?=\s?.+INTERVAL\s?=\s?.+REPORT_TYPE\s?=\s?.+', where_cause[0])
    GENERAL_ERROR = []
    if (where_cause <> None):
        all_vars = where_cause.group().split(' ')
        if len(all_vars) <> 4:
            err_str = 'The format of parameters is error.'
            if err_str not in GENERAL_ERROR:
                GENERAL_ERROR.append(err_str)
            ISOK_ALL_PARA = 0
        else:# len(all_vars) == 4
            # get value of DB_CONN_ID
            DB_CONN_ID = re.search(r'DB_CONN_ID\s?=\s?(.+)', all_vars[0])
            if DB_CONN_ID == None:
                err_str = 'The DB_CONN_ID can not be empty.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                ISOK_ALL_PARA = 0
            else:
                DB_CONN_ID = DB_CONN_ID.group(1)

            # get value of INTERVAL
            if (re.search(r'INTERVAL\s?=\s?([0-9]+$)', all_vars[2]) == None):
                err_str = 'The format of INTERVAL error.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                ISOK_ALL_PARA = 0
            else:
                INTERVAL = int(re.search(r'(\d+)', all_vars[2]).group())
                if (INTERVAL > 100):  # most get 100 data
                    INTERVAL = 100

            # get START_TIME bases on END_TIME and INTERVAL
            if (re.search(r'(\d{4}-\d{1,2}-\d{1,2})', all_vars[1]) == None) | ( re.search(r'(\d{1,2}:\d{1,2}:\d{1,2})', all_vars[1]) == None):
                err_str = 'The format of END_TIME error.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                
                ISOK_ALL_PARA = 0
            else:
                END_TIME = re.search(r'(\d{4}-\d{1,2}-\d{1,2})', all_vars[1]).group() + ' ' + re.search( r'(\d{1,2}:\d{1,2}:\d{1,2})', all_vars[1]).group()
                
                def is_valid_datetime(END_TIME):
                    try:
                        time.strptime(END_TIME, '%Y-%m-%d %H:%M:%S')
                        return True
                    except:
                        return False
                
                if is_valid_datetime(END_TIME) == True:
                    datetime_tuple = time.strptime(END_TIME, '%Y-%m-%d %H:%M:%S')
                    # To slice datetime_tuple to gain exact time data
                    year, month, day, hour, minute, second = datetime_tuple[:6]
                    final_time = datetime.datetime(year, month, day, hour, minute, second) + datetime.timedelta(hours = -INTERVAL)
                    # convert datetime fields to string
                    START_TIME = final_time.strftime('%Y-%m-%d %H:%M:%S')
                else:
                    err_str = 'The format of END_TIME error.'
                    if err_str not in GENERAL_ERROR:
                        GENERAL_ERROR.append(err_str)
                    ISOK_ALL_PARA = 0

            # get value of REPORT_TYPE
            if (re.search(r'REPORT_TYPE\s?=\s?(.+)', all_vars[3]) == None):
                err_str = 'The format of REPORT_TYPE error.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                ISOK_ALL_PARA = 0
            else:
                REPORT_TYPE = re.search(r'REPORT_TYPE\s?=\s?(.+)', all_vars[3]).group(1).upper()
                if (REPORT_TYPE != 'ALL' and REPORT_TYPE != 'RESOURCE' and REPORT_TYPE != 'CPU' and REPORT_TYPE != 'MEMORY'):
                    err_str = 'The format of REPORT_TYPE error.'
                    if err_str not in GENERAL_ERROR:
                        GENERAL_ERROR.append(err_str)
                    ISOK_ALL_PARA = 0
    else:
        err_str = 'The format of parameters is error.'
        if err_str not in GENERAL_ERROR:
            GENERAL_ERROR.append(err_str)
        ISOK_ALL_PARA = 0
        
'''
This function is to get all datatime string according to the result 
queried from database.

The variable ori_datetime_str is a dictionary and its data 
from the combination of hour_list and date_all, which will 
be used for judging whether some data exists in it or not.
'''
def get_original_datatime_str(date_all, hour_list):
    ori_datetime_str = {}
    for indx in range(len(hour_list)):
        tmp_date_str = date_all[indx].encode('unicode-escape').decode('string_escape')
        tmp_hour_str = str(hour_list[indx])
        tmp_datetime_str = ''
        if(len(tmp_hour_str) == 1):#Change 1:00:00 into 01:00:00
            tmp_hour_str = '0' + tmp_hour_str
        tmp_datetime_str = tmp_date_str + ' ' + tmp_hour_str + ':00:00'
        ori_datetime_str[tmp_datetime_str] = indx
    return ori_datetime_str

#To store previous date string value as a reference data
def format_x_axis(date_all, hour_list, x_ticks, x_ticks_lables):
    pre_date_str = str(date_all[0])

    for dateIdx in range(len(date_all)):
        # To get the data for the x-axis ticks
        x_ticks.append(float('%0.1f' % dateIdx))
        '''
        Get the data for the lable of x-axis
        If the label existed in the list x_ticks_lables,
        put hour_str into x_ticks_lables 
        otherwise,put date_lables into x_ticks_lables
        '''
        hour_str = str(x_hour_list[dateIdx])
        if len(hour_str) == 1:
            hour_str = '0' + hour_str
        date_str = str(date_all[dateIdx])
        x_lables = date_str + ' ' + hour_str
        if (dateIdx == 0):
            x_ticks_lables.append(x_lables)
        else:#dateIdx > 0
            if (pre_date_str == date_str):
                x_ticks_lables.append(hour_str)
            else:
                pre_date_str = date_str
                x_ticks_lables.append(x_lables)
                

The following sector of code is to draw graphs for Resource Consumption report according to the query result, including CPU_SEC, Logical_reads, Physical_reads, and Average Activity Time.

In [ ]:
# import the required libs
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
#To print the error of query conditon, only print once
for err_id in range(len(GENERAL_ERROR)):
    print GENERAL_ERROR[err_id]
    
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #To get data from the target database to assemble a data frame that will be used for the following graphs
    dataframe_hour = %sql select char(date(collected)) as date,hour(collected) as hours,sum(total_cpu_usec_delta)/1000000.0 as cpu_sec,sum(logical_reads_delta) as logical_reads,sum(PHYSICAL_READS_DELTA) as physical_reads,sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec,sum(act_completed_total_delta) as activities from ibm_dsm_views.throughput_all where dbconn_id='{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected),hour(collected) order by date,hours
    if dataframe_hour.empty:
        print 'For RESOURCE: The query result is empty, please check your query parameters.\n'
    else:
        #To get the hour data as x-axis from dataframe_hour
        x_hour = dataframe_hour['HOURS']
        #To get the CPU_SEC data as y-axis from dataframe_hour
        y_cpu_sec = dataframe_hour['CPU_SEC']
        y_cpu_sec_list = list(y_cpu_sec)
        
        #To get the LOGICAL_READS data as y-axis from dataframe_hour
        y_logical_reads = dataframe_hour['LOGICAL_READS']
        y_logical_reads_list = list(y_logical_reads)
       
        #To get the PHYSICAL_READS data as y-axis from dataframe_hour
        y_physical_reads = dataframe_hour['PHYSICAL_READS']
        y_physical_reads_list = list(y_physical_reads)
        
        #To get the AVG_ACTIVITY_TIME_MSEC data as y-axis from dataframe_hour
        y_avg_act = dataframe_hour['AVG_ACTIVITY_TIME_MSEC']
        y_avg_act_list = list(y_avg_act)
        
        #The tick of x-axis and sorted by asc
        x_ticks = []
        #The label of x-axis
        x_ticks_lables = []
        
        x_hour_list = list(x_hour)
        #To get date data for forming date + hour format
        date_all = list(dataframe_hour['DATE'].values)
        
        #To get original datetime for after judgement
        ori_datetime_str = get_original_datatime_str(date_all,x_hour_list)
        
        '''
        When the data queried is not equal to the requirement. Filling missing data 
        into the list x_hour_list,date_all and y_ticks.
        '''
        if (len(x_hour_list) < INTERVAL):
            #To transfer datatime data into time tuple for getting its' timestamp
            tm_tuple = time.strptime(START_TIME,'%Y-%m-%d %H:%M:%S')
            min_timestamp = time.mktime(tm_tuple)
            
            tmp_tuple = time.strptime(END_TIME,'%Y-%m-%d %H:%M:%S')
            max_timestamp = time.mktime(tmp_tuple)
            '''
            To get the difference between the max_timestamp and min_timestamp 
            which will be used for gaining all date and hour including the missing
            '''
            hour_diff = int((max_timestamp - min_timestamp)/3600)
            #clear date_all
            date_all = []
            #clear x_hour_list
            x_hour_list = []
            #clear y_ticks
            y_cpu_sec_list = []
            y_logical_reads_list = []
            y_physical_reads_list = []
            y_avg_act_list = []
            
            #Reassign the value of vars above:date_all/x_hour_list/y_cpu_sec_list/
            #y_logical_reads_list/y_physical_reads_list/y_avg_act_list
            for tmp_id in range(hour_diff):
                tmp_st = min_timestamp + tmp_id * 3600
                #To transfer timestamp to datetime string
                tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:00:00")
                tmp_date_str = tmp_datetime_str[0:10]
                tmp_hour_str = tmp_datetime_str[11:13]
                date_all.append(tmp_date_str)
                x_hour_list.append(tmp_hour_str)
                '''
                Below code is for handling y-axis's data
                If there is no data at this hour,0.0 will be filled into
                '''
                if tmp_datetime_str in ori_datetime_str:
                    tmp_index = ori_datetime_str[tmp_datetime_str]
                    y_cpu_sec_list.append(list(y_cpu_sec)[tmp_index])
                    y_logical_reads_list.append(list(y_logical_reads)[tmp_index])
                    y_physical_reads_list.append(list(y_physical_reads)[tmp_index])
                    y_avg_act_list.append(list(y_avg_act)[tmp_index])
                else:
                    y_cpu_sec_list.append(0.0)
                    y_logical_reads_list.append(0)
                    y_physical_reads_list.append(0)
                    y_avg_act_list.append(0)
            
            #To trans all list variables to numpy array
            y_cpu_sec = np.asarray(y_cpu_sec_list)
            y_logical_reads = np.asarray(y_logical_reads_list)
            y_physical_reads = np.asarray(y_physical_reads_list)
            y_avg_act = np.asarray(y_avg_act_list)
            
        #previous date string value as a reference data
        pre_date_str = str(date_all[0])
        
        #To form x_ticks and x_ticks_lables
        for dateIdx in range(len(date_all)):
            #Get the data for the x-axis ticks
            x_ticks.append(float('%0.1f' % dateIdx))
            '''
            Get the data for the lable of x-axis
            If the label existed in the list x_ticks_lables,
            put hour_str into x_ticks_lables 
            otherwise,put date_lables into x_ticks_lables
            '''
            hour_str = str(x_hour_list[dateIdx])
            if len(hour_str) == 1:
                hour_str = '0' + hour_str
            date_str = str(date_all[dateIdx])
            x_lables = date_str + ' ' + hour_str
            if (dateIdx == 0):
                x_ticks_lables.append(x_lables)
            else:#dateIdx > 0
                if (pre_date_str == date_str):
                    x_ticks_lables.append(hour_str)
                else:
                    pre_date_str = date_str
                    x_ticks_lables.append(x_lables)
        
        #To declare a Sketchpad as the first graph
        ###First graph###    
        fig = pl.figure()
        #To declare an ax container as the first drawing paper
        ax = fig.add_subplot(111)
        #To set the size of the Sketchpad depending on the size of the data
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)

        #To set the title for the first graph
        figure_title = 'CPU Usage by Hour\n'
        pl.title(figure_title,fontsize=14,fontweight='bold')
       
        #To get the DATE data as x-axis's label the data frame
        x_lable = 'Hours'
        #To set x-axis label
        pl.xlabel(x_lable)
        #To set y-axis label
        pl.ylabel(u'CPU_SEC(s)')
        #To set grid line style according to your requirement
        pl.grid(True,ls='--',color='#a6266e',alpha =0.05)
        pl.xticks(x_ticks,x_ticks_lables,rotation=90)
        
        #To mark the the value for per point
        for id in range(len(y_cpu_sec_list)):
            if(y_cpu_sec_list[id] == 0.0):#If no data,drawing a empty circle
                pl.scatter(x_ticks[id], y_cpu_sec_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
            else:
                pl.scatter(x_ticks[id], y_cpu_sec_list[id],c = '#2c628b') 
                pl.text(x_ticks[id], y_cpu_sec_list[id], '%.2f' % y_cpu_sec_list[id], fontsize = 9)
        
          
        #x_hour_isSorted=True do nothing, otherwise, handle data sperately by day
        xnew_hour = []
        ynew_cpu_sec = []
        if(len(x_hour_list) >= 3):
            xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
            #To handle the data of new y axis data
            ynew_cpu_sec = spline(np.asarray(x_ticks), y_cpu_sec,xnew_hour)
            
            ynew_cpu_sec_list = list(ynew_cpu_sec)
            #If the value of ynew_cpu_sec_list is negative, then change it into positive
            for y_idx in range(len(ynew_cpu_sec_list)):
                if (ynew_cpu_sec_list[y_idx] < 0.0):
                    ynew_cpu_sec_list[y_idx] = 0.0
            ynew_cpu_sec = np.asarray(ynew_cpu_sec_list)     
            
            #To fill the gragh according to your requirement
            pl.fill_between(xnew_hour, ynew_cpu_sec, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color='#a6266e', alpha =0.15)
            #First draw a smooth line chart using xnew_hour and ynew_memory
            pl.plot(xnew_hour, ynew_cpu_sec, color = '#a6266e')
            #To set y-axis value range according to your data
            start_value = ynew_cpu_sec.min()- ynew_cpu_sec.min() / 20
            start_value = float('%0.2f' % start_value)
            end_value = ynew_cpu_sec.max() + ynew_cpu_sec.max() / 20
            end_value = float('%0.2f' % end_value)
            pl.ylim(float('%0.2f' % start_value), float('%0.2f' % end_value))
        else:
            y_cpu_sec_list = list(map(int, y_cpu_sec_list))
            pl.yticks(y_cpu_sec_list, rotation=0)
        pl.show()

        ###Second graph###
        #To declare a variable to record wherther the value of y_logical_reads greater than 1 million or not
        log_mil = False
        if y_logical_reads.max() > 1000000:
            log_mil = True
            y_logical_reads = y_logical_reads/1000000
        
        phy_mil = False
        if y_physical_reads.max() > 1000000:
            phy_mil = True
            y_physical_reads = y_physical_reads/1000000
        
        #To declare a Sketchpad as the second graph
        fig = pl.figure()
        #To declare an ax1 container as the second drawing paper
        ax1 = fig.add_subplot(111)
        #To set the size of the Sketchpad
        if(data_size <= 20):
            fig.set_size_inches(12,3)
        elif(data_size <= 40):
            fig.set_size_inches(16,3)
        elif(data_size <= 60):
            fig.set_size_inches(18,3)
        elif(data_size <=100):
            fig.set_size_inches(22,3)

        #To set title for the second graph
        figure_title = 'I/O Consumption by Hour\n'
        pl.title(figure_title, fontsize = 14,fontweight = 'bold')
        #To set x-axis label
        pl.xlabel(x_lable, fontsize = 12)
        pl.xticks(x_ticks, x_ticks_lables, rotation=90)
        pl.bar(np.asarray(x_ticks)-0.15, y_logical_reads, width=0.3, color = '#2c628b',alpha=0.5)
        #To set y-axis label
        if (log_mil == True):
            pl.ylabel(u'LOGICAL_READS(mil times)')
        else:
            pl.ylabel(u'LOGICAL_READS(times)')
        pl.grid(True, ls = '-.', color = '#a6266e', linewidth = '.5', alpha = 0.3)
        #To set ax1 and ax2 to the same x-axis
        ax2 = ax1.twinx()

        #To draw the second graph using x_hour and y_physical_reads
        pl.bar(np.asarray(x_ticks)+0.15, y_physical_reads, width=0.3, color = '#4c78fb', alpha=0.6)
        #To set y-axis lab
        if (phy_mil==True):
            pl.ylabel(u'PHYSICAL_READS(mil times)')
        else:
            pl.ylabel(u'PHYSICAL_READS(times)')

        #To set legend for the ax1 contioner
        ax1.legend(['LOGICAL_READS'], loc='upper right', bbox_to_anchor=(0.5,1.14), ncol=1)
        #To set legend for the ax2 contioner
        ax2.legend(['PHYSICAL_READS'], loc='upper right', bbox_to_anchor=(0.7,1.14), fancybox=True, shadow=True,ncol=1)
        #To set grid format
        pl.grid(True,ls = '-.', color = '#a6266e', linewidth = '0.5', alpha=0.3)
        #To show the second graph
        pl.show()
        
        ###The third graph###
        #To declare a Sketchpad as the first graph
        fig = pl.figure()
        #To declare an ax container as the first drawing paper
        ax = fig.add_subplot(111)
        #To set the size of the Sketchpad
        if(data_size <= 20):
            fig.set_size_inches(12,2)
        elif(data_size <= 40):
            fig.set_size_inches(16,2)
        elif(data_size <= 60):
            fig.set_size_inches(18,2)
        elif(data_size <= 100):
            fig.set_size_inches(22,2)

        #To set the title for the first graph
        figure_title = 'Average Activity Time by Hour'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
       
        #To set x-axis label
        pl.xlabel(x_lable, fontsize = 12)
        #To set y-axis label
        pl.ylabel(u'AVG_ACTIVITY_TIME_MSEC', fontsize = 9)
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color= '#a6266e', alpha = 0.05)
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        
        #To form x_ticks and x_ticks_lables
        for id in range(len(y_avg_act_list)):
            if(y_avg_act_list[id] == 0.0):#If no data,drawing a empty circle
                pl.scatter(x_ticks[id], y_avg_act_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
            else:
                pl.scatter(x_ticks[id], y_avg_act_list[id], c = '#2c628b') 
                pl.text(x_ticks[id], y_avg_act_list[id], '%.0f' % y_avg_act_list[id], fontsize = 9)
        
        #To smooth the line
        xnew_hour = []
        ynew_cpu_sec = []
        if(len(x_hour_list) >= 3):
            xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
            #To handle the data of new y axis data
            ynew_avg_act = spline(np.asarray(x_ticks), y_avg_act, xnew_hour)
            ynew_avg_act_list = list(ynew_avg_act)
            #If the value of ynew_cpu_sec_list is negative, then change it into positive
            for y_idx in range(len(ynew_avg_act_list)):
                if (ynew_avg_act_list[y_idx] < 0.0):
                    ynew_avg_act_list[y_idx] = 0.0
            ynew_avg_act = np.asarray(ynew_avg_act_list) 
            #Fill the gragh according to your requirement
            pl.fill_between(xnew_hour,ynew_avg_act,where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#a6266e', alpha = 0.15)
            #First draw a smooth line chart using xnew_hour and ynew_memory
            pl.plot(xnew_hour, ynew_avg_act, color = '#a6266e')
            #Set y-axis value range according to your data
            start_value = ynew_avg_act.min() - ynew_avg_act.min() / 2
            start_value = float('%0.2f' % start_value)
            end_value = ynew_avg_act.max() + ynew_avg_act.max() / 2
            end_value = float('%0.2f' % end_value)
            pl.ylim(float('%0.2f' % start_value), float('%0.2f' % end_value))
        else:
            y_avg_act_list = list(map(int, y_avg_act_list))
            pl.yticks(y_avg_act_list, rotation = 0)
        pl.show()

The following sector of code is to draw a graph for CPU Utilization report according to the query result. 

In [ ]:
#import the required libs
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
    
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'CPU')):
    #To get data from the target database to assemble a data frame that will be used for the following graphs
    dataframe_hour = %sql select char(date(collected)) as date, hour(collected) as hours, sum(db2_cpu_user_delta) as db2_cpu_user,  100*sum(db2_cpu_user_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as db2_cpu_user_percent, sum(db2_cpu_system_delta) as db2_cpu_system,  100*sum(db2_cpu_system_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as db2_cpu_system_percent,sum(cpu_user_delta) as cpu_user,  100*sum(cpu_user_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as cpu_user_percent,sum(cpu_system_delta) as cpu_system,  100*sum(cpu_system_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as cpu_system_percent from IBM_DSM_VIEWS.THROUGHPUT_SYSTEM where dbconn_id='{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected),hour(collected) order by date,hours
    if dataframe_hour.empty:
        print 'For CPU: The query result is empty, please check your query parameters.\n'
    else:
        #To get the hour data as x-axis from the data frame
        x_hour = dataframe_hour['HOURS']
        #To get the db2_cpu_user_percent data as top y-axis of the first graph from the data frame
        y_db2_user = dataframe_hour['DB2_CPU_USER_PERCENT']
        y_db2_user_list = []
        #To get the db2_cpu_system_percent data as bottom y-axis of the first graph from the data frame
        y_db2_sys = dataframe_hour['DB2_CPU_SYSTEM_PERCENT']
        y_db2_sys_list = []
        #To get the cpu_user_percent data as top y-axis of the second graph from the data frame
        y_cpu_user = dataframe_hour['CPU_USER_PERCENT']
        y_cpu_user_list = []
        #To get the cpu_system_percent data as bottom y-axis of the second graph from the data frame
        y_cpu_sys = dataframe_hour['CPU_SYSTEM_PERCENT']
        y_cpu_sys_list = []
        
        #The tick of x-axis and sorted by asc
        x_ticks = []
        #The label of x-axis
        x_ticks_lables = []
        
        x_hour_list = list(x_hour)
        #To get date data for forming date + hour format
        date_all = list(dataframe_hour['DATE'].values)
        #To get original datetime used by following judgement
        ori_datetime_str = get_original_datatime_str(date_all, x_hour_list)
        
        '''
        When the data queried is not equal to the requirement. Filling missing data 
        into the list x_hour_list,date_all and y_ticks.
        '''
        if (len(x_hour_list) < INTERVAL):
            #To transfer datatime data into time tuple for getting its' timestamp
            tm_tuple = time.strptime(START_TIME, '%Y-%m-%d %H:%M:%S')
            min_timestamp = time.mktime(tm_tuple)
            
            tmp_tuple = time.strptime(END_TIME, '%Y-%m-%d %H:%M:%S')
            max_timestamp = time.mktime(tmp_tuple)            

            '''
            To get the difference between the max_timestamp and min_timestamp 
            which will be used for gaining all date and hour including the missing
            '''
            hour_diff = int((max_timestamp - min_timestamp) / 3600)
            #clear date_all
            date_all = []
            #clear x_hour_list
            x_hour_list = []
            
            #Reassign above vaiables:date_all / x_hour_list
            for tmp_id in range(hour_diff):
                tmp_st = min_timestamp + tmp_id * 3600
                #To change timestamp to datetime string
                tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:00:00")
                tmp_date_str = tmp_datetime_str[0:10]
                tmp_hour_str = tmp_datetime_str[11:13]
                date_all.append(tmp_date_str)
                x_hour_list.append(tmp_hour_str)
                '''
                Below code is for handling y-axis's data
                If there is no data at this hour,0.0 will be filled into
                '''
                if tmp_datetime_str in ori_datetime_str:
                    tmp_index = ori_datetime_str[tmp_datetime_str]
                    y_db2_user_list.append(list(y_db2_user)[tmp_index])
                    y_db2_sys_list.append(list(y_db2_sys)[tmp_index])
                    y_cpu_user_list.append(list(y_cpu_user)[tmp_index])
                    y_cpu_sys_list.append(list(y_cpu_sys)[tmp_index])
                else:
                    y_db2_user_list.append(0)
                    y_db2_sys_list.append(0)
                    y_cpu_user_list.append(0)
                    y_cpu_sys_list.append(0)
            y_db2_user = np.asarray(y_db2_user_list)
            y_db2_sys = np.asarray(y_db2_sys_list)
            y_cpu_user = np.asarray(y_cpu_user_list)
            y_cpu_sys = np.asarray(y_cpu_sys_list)
            
        #previous date string value as a reference data
        pre_date_str = str(date_all[0])
        
        for dateIdx in range(len(date_all)):
            #To get the data for the x-axis ticks
            x_ticks.append(float('%0.1f' % dateIdx))
            '''
            Get the data for the lable of x-axis
            If the label existed in the list x_ticks_lables,
            put hour_str into x_ticks_lables 
            otherwise,put date_lables into x_ticks_lables
            '''
            hour_str = str(x_hour_list[dateIdx])
            if len(hour_str) == 1:
                hour_str = '0' + hour_str
            date_str = str(date_all[dateIdx])
            x_lables = date_str + ' ' + hour_str
            if (dateIdx == 0):
                x_ticks_lables.append(x_lables)
            else:#dateIdx > 0
                if (pre_date_str == date_str):
                    x_ticks_lables.append(hour_str)
                else:
                    pre_date_str = date_str
                    x_ticks_lables.append(x_lables)
        
        #To declare a Sketchpad including two graphs of left and right distribution
        fig = pl.figure()
        
        #To declare an ax1 container as the left drawing paper
        ax1 = fig.add_subplot(111)
        #To set the size of the Sketchpad
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)
        #To set the title for the left graph
        figure_title = 'CPU Utilization by Hour\n'
        pl.title(figure_title, fontsize=14, fontweight='bold')
        
        pl.xticks(x_ticks, x_ticks_lables, rotation=90)
       
        #To set y-axis label
        pl.ylabel(u'CPU %')
        #To set x-axis label
        x_lable= 'Hours'
        pl.xlabel(x_lable)
        x_hour = np.asarray(x_ticks)
        #To draw the top y-axis of the upper graph
        pl.bar(x_hour - 0.2, y_db2_user, width = 0.4, align = 'center', color = '#60bdae')
        #To draw the bottom y-axis of the upper graph
        pl.bar(x_hour - 0.2, y_db2_sys, width=0.4, align = 'center', color = '#a6266e',bottom = y_db2_user)
        #To draw the top y-axis of the lower graph
        pl.bar(x_hour + 0.2, y_cpu_user, width=0.4, color = '#4c78fb')
        #To draw the bottom y-axis of the lower graph
        pl.bar(x_hour + 0.2, y_cpu_sys, width=0.4, color = '#2c628b', bottom = y_cpu_user)
        #To set the y-axis labels
        pl.yticks(np.arange(0, 101, 10))

        #To set the legends for the both graphs
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
        ax1.legend(['DB2_CPU_USER_PERCENT', 'DB2_CPU_SYSTEM_PERCENT', 'CPU_USER_PERCENT', 'CPU_SYSTEM_PERCENT'], fontsize = 9, loc = 'upper center', bbox_to_anchor=(0.5,1.06), fancybox = True, shadow = True, ncol = 4)
        #To set grid format for ax1
        pl.grid(True, ls = '-.', color = '#a6266e', linewidth = '0.2', alpha = 0.3)

        #To Show graphs
        pl.show()

The following sector of code is to draw a graph for Memory Usage report according to the query result.

In [ ]:
#import the required lib
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
    
#To get data and draw graph by data.
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'MEMORY')):
    #Get data from the target database to assemble a data frame that will be used for the following graph
    dataframe_hour = %sql select char(date(collected)) as date,hour(collected) as hours,dec(max(memory_pool_used_gb),17,2) as memory_usage_gb from IBM_DSM_VIEWS.MEM_DB_TOTAL_USED where dbconn_id='{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected),hour(collected) order by date,hours
    
    if dataframe_hour.empty:
        print 'For MEMORY: The query result is empty, please check your query parameters.\n'
    else:
        #To get the hour data as x-axis from the data frame and convert the data into a numpy array
        x_hour = dataframe_hour['HOURS'].values #here x_hour is a numpy narray
        #To get the memory_usage_gb data as y-axis from the data frame and convert the data into a numpy array
        y_memory = dataframe_hour['MEMORY_USAGE_GB'].values
        #To define a empty var x_ticks to store x-axis ticks(marks)
        x_ticks = []
        #To define a empty var x_ticks_lables to restore x-axis labels
        x_ticks_lables = []
        #To define a empty var y_ticks to store y-axis ticks(marks)
        y_memory_list = list(y_memory)
        
        x_hour_list = list(x_hour)
        date_all = list(dataframe_hour['DATE'].values)
        
        #To get original datetime used by following judgement
        ori_datetime_str = get_original_datatime_str(date_all, x_hour_list)
        
        '''
        When the data queried is not equal to the requirement. Filling missing data 
        into the list x_hour_list,date_all and y_memory_list.
        '''
        if (len(x_hour_list) < INTERVAL):
            #To transfer datatime data into time tuple for getting its' timestamp
            tm_tuple = time.strptime(START_TIME, '%Y-%m-%d %H:%M:%S')
            min_timestamp = time.mktime(tm_tuple)
            
            tmp_tuple = time.strptime(END_TIME, '%Y-%m-%d %H:%M:%S')
            max_timestamp = time.mktime(tmp_tuple)
            
            '''
            Get the difference between the max_timestamp and min_timestamp 
            which will be used for gaining all date and hour including the missing
            '''
            hour_diff = int((max_timestamp - min_timestamp) / 3600)
            # To clear date_all
            date_all = []
            # To clear x_hour_list
            x_hour_list = []
            # To declare a new list to store y_memory data temporarily
            tmp_y_ticks = []
            
            #To reassign three variable above:date_all / x_hour_list / tmp_y_ticks
            for tmp_id in range(hour_diff):
                tmp_st = min_timestamp + tmp_id * 3600
                #To transfer timestamp to datetime string
                tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:00:00")
                tmp_date_str = tmp_datetime_str[0:10]
                tmp_hour_str = tmp_datetime_str[11:13]
                date_all.append(tmp_date_str)
                x_hour_list.append(tmp_hour_str)
                '''
                Below code is for handling y-axis's data
                If there is no data at this hour,0.0 will be filled into
                '''
                if tmp_datetime_str in ori_datetime_str:
                    tmp_index = ori_datetime_str[tmp_datetime_str]
                    tmp_y_ticks.append(y_memory_list[tmp_index])
                else:
                    tmp_y_ticks.append(0.0)
            y_memory_list = tmp_y_ticks
            
        #To store previous date string value as a reference data
        pre_date_str = str(date_all[0])
        
        for dateIdx in range(len(date_all)):
            # To get the data for the x-axis ticks
            x_ticks.append(float('%0.1f' % dateIdx))
            '''
            Get the data for the lable of x-axis
            If the label existed in the list x_ticks_lables,
            put hour_str into x_ticks_lables 
            otherwise,put date_lables into x_ticks_lables
            '''
            hour_str = str(x_hour_list[dateIdx])
            if len(hour_str) == 1:
                hour_str = '0' + hour_str
            date_str = str(date_all[dateIdx])
            x_lables = date_str + ' ' + hour_str
            if (dateIdx == 0):
                x_ticks_lables.append(x_lables)
            else:#dateIdx > 0
                if (pre_date_str == date_str):
                    x_ticks_lables.append(hour_str)
                else:
                    pre_date_str = date_str
                    x_ticks_lables.append(x_lables)
        #To declarea Sketchpad
        fig = pl.figure()
        #To declare an ax container as a drawing paper
        ax = fig.add_subplot(111)
        #To set the size of the Sketchpad depending on the size of the data
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)
            
        #To set the title/label/grid for the graph
        figure_title = 'Memory Usage by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        x_lable = 'Hours'
        #To set x-axis label
        pl.xlabel(x_lable)
        #To set y-axis label
        pl.ylabel(u'MEMORY_USAGE(GB)')
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        
        #To mark the data point 
        for id in range(len(y_memory_list)):
            if(y_memory_list[id] == 0.0):#If no data,drawing a empty circle
                pl.scatter(x_ticks[id], y_memory_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
            else:
                pl.scatter(x_ticks[id], y_memory_list[id], c = '#2c628b') 
                pl.text(x_ticks[id], y_memory_list[id], '%.2f' % y_memory_list[id], fontsize = 9)
        
        #Below two variable is used for storeing magnified data
        xnew_hour = []
        ynew_memory = []
        '''
        if the size of data is greater than 2, showing a line not a scatter
        otherwise, a scatter graph will be showed
        '''
        if(len(x_hour_list) >= 3):
            ##In order to smooth the line chart,handle the data further##
            #Expand each x axis data 20 times
            xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
            #To handle the data of new y axis data
            ynew_memory= spline(np.asarray(x_ticks), np.asarray(y_memory_list), xnew_hour)
            ynew_memory_list = list(ynew_memory)
            #If the value of ynew_cpu_sec_list is negative, then change it into positive
            for y_idx in range(len(ynew_memory_list)):
                if (ynew_memory_list[y_idx] < 0.0):
                    ynew_memory_list[y_idx] = 0.0
            ynew_memory = np.asarray(ynew_memory_list)        
            #To fill the gragh according to your requirement
            pl.fill_between(xnew_hour, ynew_memory, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#2c628b', alpha = 0.09)
            #To draw curve graph
            pl.plot(xnew_hour, ynew_memory, color = '#2c628b')
            #To set y-axis value range according to your data
            start_value=ynew_memory.min() -  ynew_memory.min() / 2
            start_value=float('%.2f' % start_value)
            end_value=ynew_memory.max() + ynew_memory.max() / 2
            end_value=float('%.2f' % end_value)
            #To set the scale for y-axis
            pl.ylim(float('%.2f' % start_value),float('%.2f' % end_value))
            #pl.yticks(np.linspace(start_value,end_value,0.1,endpoint=True))
        pl.show()